In [1]:
!pip install transformers

In [2]:
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import random
import numpy as np
from tqdm import tqdm, trange
import time
import logging
# logging.basicConfig(level=logging.INFO)
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from sklearn.metrics import f1_score, precision_score, recall_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pd.set_option('max_colwidth', 400)
print(device)

cuda


In [0]:
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)
timestamp = time.strftime("%Y.%m.%d_%H.%M.%S", time.localtime())
fh = logging.FileHandler('log_model.txt')
fh.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] ## %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(ch)

In [0]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything()

In [5]:
# Applying a first round of text cleaning techniques
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
import nltk
import re
import string
from bs4 import BeautifulSoup
from textblob import TextBlob

tokenizer = nltk.tokenize.TweetTokenizer(
        strip_handles=True, reduce_len=True)

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def clean_text_no_smiley(text):
    text = BeautifulSoup(text, 'lxml').get_text()
    eyes = "[8:=;]"
    nose = "['`\-]?"
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " ", text)

    text = re.sub("/", " / ", text)
    text = re.sub('@(\w+)', '', text)

    text = re.sub('#{eyes}#{nose}[)d]+|[)d]+#{nose}#{eyes}', " ", text)
    text = re.sub('#{eyes}#{nose}p+', " ", text)
    text = re.sub('#{eyes}#{nose}\(+|\)+#{nose}#{eyes}', " ", text)
    text = re.sub('#{eyes}#{nose}[\/|l*]', " ", text)
    text = re.sub('<3', " ", text)
    # numbers
    text = re.sub('[-+]?[.\d]*[\d]+[:,.\d]*', " ", text)

    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    # text = re.sub('\[.*?\]', '', text)
    # text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(
        string.punctuation.replace("'", "")), ' ', text)
    text = re.sub('\n', ' ', text)
    text = ''.join(filter(lambda x: x in string.printable, text))
    # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    # text = re.sub('\w*\d\w*', '', text)

    return text


def text_preprocessing_no_lemmatizer(text):

    nopunc = clean_text_no_smiley(text)

    tokenized_text = tokenizer.tokenize(nopunc)

    combined_text = ' '.join(tokenized_text)
    return combined_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
try:
    train = pd.read_csv('input/preprocessed_train.csv')
    print('Training data shape: ', train.shape)
    test = pd.read_csv('input/preprocessed_test.csv')
    print('Testing data shape: ', test.shape)
except:
    train = pd.read_csv('drive/My Drive/NLP_data/train.csv')
    print('Training data shape: ', train.shape)
    test = pd.read_csv('drive/My Drive/NLP_data/test.csv')
    print('Testing data shape: ', test.shape)

    train['text'] = train['text'].apply(
        lambda x: text_preprocessing_no_lemmatizer(x))
    test['text'] = test['text'].apply(
        lambda x: text_preprocessing_no_lemmatizer(x))

    train.drop(["keyword", "location"], axis=1, inplace=True)
    test.drop(["keyword", "location"], axis=1, inplace=True)

    train.to_csv('input/preprocessed_train.csv')
    test.to_csv('input/preprocessed_test.csv')


train[['text']].sample(5)

Training data shape:  (7613, 4)
Testing data shape:  (3263, 3)


,text
2644,so you have new weapon that can cause un imaginable destruction
2227,the things do for gishwhes just got soaked in deluge going for pads and tampons thx
5448,dt rt the col police can catch pickpocket in liverpool stree
132,aftershock back to school kick off was great want to thank everyone for making it possible what great night
6845,in response to trauma children of addicts develop defensive self one that decreases vulnerability


In [7]:
bertTokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
max_len = 0

# For every sentence...
for sentence in train["text"]:
    input_ids = bertTokenizer.encode(sentence, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  43


In [0]:
max_len = 64

In [0]:
def convert_df_to_BERT_input(sequences, bertTokenizer):
    input_ids = []
    segment_ids = []
    attention_masks = []
  
    for index, sequence in enumerate(sequences):
        # Split the sentence into tokens.
        encoded_dict = bertTokenizer.encode_plus(sequence, add_special_tokens=True, max_length=max_len, 
                                              pad_to_max_length=True,
                                              return_attention_mask = True,
                                              return_token_type_ids = True,
                                              return_tensors = 'pt',     # Return pytorch tensors.
                                              )   
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        segment_ids.append(encoded_dict['token_type_ids'])
        
    return (input_ids, attention_masks, segment_ids)

In [0]:
input_ids, attention_masks, segment_ids = convert_df_to_BERT_input(train['text'], bertTokenizer)

In [11]:
print(input_ids[0])
print(segment_ids[0])
print(attention_masks[0])

tensor([[  101,  2256, 15616,  2024,  1996,  3114,  1997,  2023,  8372,  2089,
         16455,  9641,  2149,  2035,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [0]:
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
segment_ids = torch.cat(segment_ids, dim=0)
labels = torch.tensor(train['target'], dtype=torch.long) 

In [13]:
print(input_ids[0])
print(segment_ids[0])
print(attention_masks[0])
print(labels[0])

tensor([  101,  2256, 15616,  2024,  1996,  3114,  1997,  2023,  8372,  2089,
        16455,  9641,  2149,  2035,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor(1)


In [0]:
dataset = TensorDataset(input_ids, attention_masks, segment_ids, labels)

In [0]:
from sklearn.model_selection import train_test_split

train_dataset, validation_dataset = train_test_split(
   dataset, test_size=0.1, random_state=1000)

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            validation_dataset, # The validation samples.
            sampler = SequentialSampler(validation_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [17]:
train_dataloader

In [19]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model_type = "bert-large-uncased"
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    model_type, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2,
    output_attentions = False, 
    output_hidden_states = False
)

# Tell pytorch to run this model on the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [0]:
from transformers import get_linear_schedule_with_warmup
learning_rate = 5e-5 #, 3e-5, 2e-5
epochs = 3 # 2, 3, 4
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

### Train

In [21]:

train_steps = 0
model.zero_grad()
model.train()
train_iterator = trange(int(epochs), desc="Epoch")
for epoch in train_iterator:
  tr_loss = 0.0
  preds = None
  out_label_ids = None
  model.train()

  for step, batch in enumerate(train_dataloader):
    
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2],
              'labels': batch[3]}    
    outputs = model(**inputs)
    loss, logits = outputs[:2]  # model outputs are always tuple in transformers (see doc)
    loss.backward()
    tr_loss += loss.item()
    train_steps += 1
    optimizer.step()
    scheduler.step()  # Update learning rate schedule
    model.zero_grad()
    if preds is None:
      preds = logits.detach().cpu().numpy()
      out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
      preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
      out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
    
  # Calculate the average loss over all of the batches.
  avg_train_loss = tr_loss / len(train_dataloader)            
  preds = np.argmax(preds, axis=1)  
  print("")
  print(" Epoch {0:.2f} Average training loss: {0:.2f}".format(epoch, avg_train_loss))
  print("          F1 score: {0:.2f}".format(f1_score(out_label_ids, preds)))

  #########################################################
  # Validate
  eval_loss = 0.0
  nb_eval_steps = 0
  preds = None
  out_label_ids = None
  for batch in validation_dataloader:
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
      inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels':         batch[3]}
      outputs = model(**inputs)
      tmp_eval_loss, logits = outputs[:2]
      eval_loss += tmp_eval_loss.mean().item()
  
    nb_eval_steps += 1

    if preds is None:
      preds = logits.detach().cpu().numpy()
      out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
      preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
      out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)


  avg_eval_loss = eval_loss / nb_eval_steps
  preds = np.argmax(preds, axis=1)
    
  print("")
  print(" Average validation loss: {0:.2f}".format(avg_eval_loss))
  print(" F1 score: {0:.2f}".format(f1_score(out_label_ids, preds)))
   

      


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]


 Epoch 0.00 Average training loss: 0.00
          F1 score: 0.76


Epoch:  33%|███▎      | 1/3 [04:10<08:21, 250.58s/it]


 Average validation loss: 0.36
 F1 score: 0.81

 Epoch 1.00 Average training loss: 1.00
          F1 score: 0.86


Epoch:  67%|██████▋   | 2/3 [08:27<04:12, 252.37s/it]


 Average validation loss: 0.40
 F1 score: 0.81

 Epoch 2.00 Average training loss: 2.00
          F1 score: 0.91


Epoch: 100%|██████████| 3/3 [12:43<00:00, 254.65s/it]


 Average validation loss: 0.47
 F1 score: 0.80


## Predict

In [0]:
input_ids, attention_masks, segment_ids = convert_df_to_BERT_input(test['text'], bertTokenizer)

In [23]:
print(test['text'][0])
print(input_ids[0])
print(segment_ids[0])
print(attention_masks[0])

just happened terrible car crash
tensor([[ 101, 2074, 3047, 6659, 2482, 5823,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [0]:
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
segment_ids = torch.cat(segment_ids, dim=0)

In [25]:
print(input_ids[0])
print(segment_ids[0])
print(attention_masks[0])

tensor([ 101, 2074, 3047, 6659, 2482, 5823,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [0]:
test_dataset = TensorDataset(input_ids, attention_masks, segment_ids)

In [0]:
test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size
        )

In [0]:
test_steps = 0
preds = None
out_label_ids = None
for batch in test_dataloader:
  model.eval()
  batch = tuple(t.to(device) for t in batch)
  with torch.no_grad():
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'token_type_ids': batch[2]
              }
    outputs = model(**inputs)
    logits = outputs[0]
  
  test_steps += 1

  if preds is None:
    preds = logits.detach().cpu().numpy()
  else:
    preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

preds = np.argmax(preds, axis=1)

In [29]:
print(test_steps)
submission_file_path = "drive/My Drive/NLP_data/sample_submission.csv"

sample_submission = pd.read_csv(submission_file_path)

sample_submission["target"] = preds
sample_submission.to_csv("submission.csv", index=False)

102


In [95]:
print(sample_submission["target"][:20])

0     1
1     1
2     1
3     1
4     1
5     1
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    1
16    0
17    1
18    0
19    0
Name: target, dtype: int64
